In [1]:
import os
from semantic_kernel.agents import AssistantAgentThread, AzureAssistantAgent
from azure.ai.agents.models import CodeInterpreterTool
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread
from azure.identity import EnvironmentCredential, get_bearer_token_provider
from dotenv import load_dotenv

load_dotenv()
pool_management_endpoint = os.getenv("POOL_MANAGEMENT_ENDPOINT")
ai_project_endpoint = os.getenv("AI_PROJECT_ENDPOINT")
chat_endpoint=os.getenv("OPENAI_ENDPOINT")
model_deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME")

## Semantic Kernel + AOAI Assistants API

In [6]:
import time
from datetime import timedelta
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

# Custom polling strategy
polling = RunPollingOptions(
    run_polling_interval=timedelta(seconds=1),          # base wait between GET /runs
    run_polling_backoff=timedelta(seconds=2),           # how much to add after N tries
    run_polling_backoff_threshold=4,                    # after 4 polls, start backing off
    maximum_retry_count=60                              # give up after 60 polls  (~90 s)
)
# 1. Create the client using Azure OpenAI resources and configuration
provider = get_bearer_token_provider(EnvironmentCredential(), "https://cognitiveservices.azure.com/.default")
start_time = time.time()
client = AzureAssistantAgent.create_client(
    endpoint=chat_endpoint, 
    ad_token_provider=provider, 
    deployment_name=model_deployment_name,
    api_version="2025-04-01-preview"
)

# 2. Configure the code interpreter tool and resources for the Assistant
code_interpreter_tool, code_interpreter_tool_resources = AzureAssistantAgent.configure_code_interpreter_tool()

async def create_agent() -> bool:
    # 3. Create the assistant on the Azure OpenAI service
    definition = await client.beta.assistants.create(
        model=model_deployment_name,
        name="CodeRunner",
        instructions="Run the provided request as code and return the result. ALWAYS USE THE CODE INTERPRETER TOOL.",
        tools=code_interpreter_tool,
        tool_resources=code_interpreter_tool_resources
    )

    # 4. Create a Semantic Kernel agent for the Azure OpenAI assistant
    agent = AzureAssistantAgent(
        client=client,
        definition=definition,
        polling=polling,
    )

    # 5. Create a new thread for use with the assistant
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AssistantAgentThread = None

    TASK = """Use the python code interpreter, 
    print the current python version
    then print the current date and time
    then determine the values in the Fibonacci sequence that that are less than the value of 101?"""

    # print(f"# User: '{TASK}'")
    success = False
    try:
        # 6. Invoke the agent for the current thread and print the response
        async for response in agent.invoke(messages=TASK, thread=thread):
            # print(f"# Agent: {response}")
            thread = response.thread
            success = True
    finally:
        # 7. Clean up the resources
        await thread.delete() if thread else None
        await agent.client.beta.assistants.delete(agent.id)
        return success
        
import asyncio

async def run_agent_instance(idx) -> bool:
    success = await create_agent()
    return success

async def run_agents_in_parallel():
    tasks = [run_agent_instance(i) for i in range(100)]
    results = await asyncio.gather(*tasks)
    print(results)
    # results is a list of booleans, count the number of True and False
    num_succeeded = results.count(True)
    num_failed = results.count(False)
    print(f"Total succeeded: {num_succeeded}")
    print(f"Total failed: {num_failed}")

# To actually run the agents in parallel, you would call:
await run_agents_in_parallel()


[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
Total succeeded: 100
Total failed: 0


## Semantic Kernel + AI Foundry Agent SDK

Authentication is also handled with service principals - just need to ensure the SPN has a role assignment to the resource with the `Azure AI User` role.

In [ ]:
# 1. Create client with connection string and auth to AI Foundry project
client = AzureAIAgent.create_client(endpoint=ai_project_endpoint, credential=EnvironmentCredential())

# 2. Create an agent with a code interpreter on the Azure AI agent service
code_interpreter = CodeInterpreterTool(file_ids=[])
agent_definition = await client.agents.create_agent(
    model=model_deployment_name,
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
    instructions="Run the provided request as code and return the result. ALWAYS USE THE CODE INTERPRETER TOOL.",
)

# 3. Create a Semantic Kernel agent for the Azure AI agent
agent = AzureAIAgent(
    client=client,
    definition=agent_definition,
)

# 4. Create a thread for the agent
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: AzureAIAgentThread | None = None

TASK = """Use the python code interpreter, 
print the current python version
then print the current date and time
then determine the values in the Fibonacci sequence that that are less than the value of 101?"""
print(f"# User: '{TASK}'")

try:
    # 5. Invoke the agent for the specified thread for response
    async for response in agent.invoke(messages=TASK, thread=thread):
        print(f"# Agent: {response}")
        thread = response.thread
finally:
    # 6. Cleanup: Delete the thread, agent, and file
    await thread.delete() if thread else None
    await client.agents.delete_agent(agent.id)
